# Assignment-6

**Task:** Main objective is to construct a stock portfolio using stocks listed in S&P-500. To do that, you will pick 50 stocks from 5 different sectors (10 stocks from each sector). You ultimate aim is to run PCA for each sector and see which stock dominates the sector and to run performance analysis using Sharpe ratio.

1) Retrieve daily data covering the period of 01/01/2017-01/01/2022.

2) Calculate returns of your stocks

3) Run PCA for different sectors. If you end up with more than one principal component, calculate cross sectional average of these components to have only one variable.

4) Calculate the loadings to show which stock dominates the sector (loadings gives the weight of the stock in a portfolio)

5) Calculate sharpe ratio for each sector and compare it to the Sharpe ratio of S&P-500 for the same period.


In [82]:
#codes here
import yfinance as yf
import datetime
import pandas as pd
from fredapi import Fred
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [83]:
#Chevron, ExxonMobil, Hess, Baker Hughes, Halliburton, Schlumberger, APA Corporation, ConocoPhillips, Coterra, Devon
energy_stocks = ['CVX', 'XOM', 'HES', 'BKR', 'HAL', 'SLB', 'APA', 'COP', 'CTRA', 'DVN']
#Blackrock, BNY Mellon, Franklin Templeton, Invesco, Nothern Trust, State Street, T Rowe Price, American Express, Capitol One, Discover
financial_stocks = ['BLK', 'BK', 'BEN', 'IVZ', 'NTRS', 'STT', 'TROW', 'AXP', 'COF', 'DFS'] 
#Amgen, Biogen, Gilead, Incyte, Moderna, Regeneron, Vertex, AmerisourceBergen, Bristol Myers Squibb, Cardinal Health
health_stocks = ['AMGN', 'BIIB', 'GILD', 'INCY', 'MRNA', 'REGN', 'VRTX', 'ABC', 'BMY', 'CAH']
#Boeing, General Dynamics, Lockheed Martin, Raytheon Technologies, Fedex, Alaska Airlines, American Airlines, Carrier, Masco, Jacobs
industrial_stocks = ['BA', 'GD', 'LMT', 'RTX', 'FDX', 'ALK', 'AAL', 'CARR', 'MAS', 'J']
#Adobe, Visa, Oracle, Salesforce, Mastercard, AMD, Nvidia, Microsoft, Western Digital, Apple
IT_stocks = ['ADBE', 'V', 'ORCL', 'CRM', 'MA', 'AMD', 'NVDA', 'MSFT', 'WDC', 'AAPL']

start = datetime.datetime(2017, 1, 1)
end = datetime.datetime(2022, 1, 1)

In [84]:
energy_prices = yf.download(energy_stocks, start = start, end = end, interval = '1d')
financial_prices = yf.download(financial_stocks, start = start, end = end, interval = '1d')
health_prices = yf.download(health_stocks, start = start, end = end, interval = '1d')
industrial_prices = yf.download(industrial_stocks, start = start, end = end, interval = '1d')
IT_prices = yf.download(IT_stocks, start = start, end = end, interval = '1d')
#S&P 500
SP_prices = yf.download(['^GSPC'], start = start, end = end, interval = '1d')


[*********************100%***********************]  10 of 10 completed
[*********************100%***********************]  10 of 10 completed
[*********************100%***********************]  10 of 10 completed
[*********************100%***********************]  10 of 10 completed
[*********************100%***********************]  10 of 10 completed
[*********************100%***********************]  1 of 1 completed


In [85]:
energy_prices_close = energy_prices['Close']
financial_prices_close = financial_prices['Close']
health_prices_close = health_prices['Close']
industrial_prices_close = industrial_prices['Close']
IT_prices_close = IT_prices['Close']
SP_prices_close = SP_prices['Close']
energy_prices_close.dropna(inplace = True)
financial_prices_close.dropna(inplace = True)
health_prices_close.dropna(inplace = True)
industrial_prices_close.dropna(inplace = True)
IT_prices_close.dropna(inplace = True)
SP_prices_close.dropna(inplace = True)

C:\Users\Peter\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [86]:
fred = Fred(api_key = '259593b221ea34aa8f3c5cb39ab85c31')
risk_free = fred.get_series('DGS3MO')
risk_free = risk_free['2017-01-01':'2022-01-01']
risk_free = risk_free / 90

In [87]:
energy_returns = energy_prices_close.pct_change()
financial_returns = financial_prices_close.pct_change()
health_returns = health_prices_close.pct_change()
industrial_returns = industrial_prices_close.pct_change()
IT_returns = IT_prices_close.pct_change()
SP_returns = pd.DataFrame(SP_prices_close.pct_change())
energy_returns.dropna(inplace = True)
financial_returns.dropna(inplace = True)
health_returns.dropna(inplace = True)
industrial_returns.dropna(inplace = True)
IT_returns.dropna(inplace = True)
SP_returns.dropna(inplace = True)
SP_returns = SP_returns.rename({'Close':'GSPC'}, axis='columns')

In [88]:
for i in energy_returns:
    energy_returns[f'excess_return_{i}'] = energy_returns[i] - risk_free
for i in financial_returns:
    financial_returns[f'excess_return_{i}'] = financial_returns[i] - risk_free
for i in health_returns:
    health_returns[f'excess_return_{i}'] = health_returns[i] - risk_free
for i in industrial_returns:
    industrial_returns[f'excess_return_{i}'] = industrial_returns[i] - risk_free
for i in IT_returns:
    IT_returns[f'excess_return_{i}'] = IT_returns[i] - risk_free
for i in SP_returns:
    SP_returns[f'excess_return_{i}'] = SP_returns[i] - risk_free

In [89]:
ER = energy_returns.iloc[:, -10:].values
FR = financial_returns.iloc[:, -10:].values
HR = health_returns.iloc[:, -10:].values
IR = industrial_returns.iloc[:, -10:].values
ITR = IT_returns.iloc[:, -10:].values

In [90]:
ss_ER = StandardScaler().fit_transform(ER)
ss_FR = StandardScaler().fit_transform(FR)
ss_HR = StandardScaler().fit_transform(HR)
ss_IR = StandardScaler().fit_transform(IR)
ss_ITR = StandardScaler().fit_transform(ITR)

In [91]:
n_ER = ss_ER[np.logical_not(np.isnan(ss_ER))]
n_FR = FR[np.logical_not(np.isnan(FR))]
n_HR = HR[np.logical_not(np.isnan(HR))]
n_IR = IR[np.logical_not(np.isnan(IR))]
n_ITR = ITR[np.logical_not(np.isnan(ITR))]

In [92]:
ER2 = np.reshape(n_ER, (1249, 10))
FR2 = np.reshape(n_FR, (1249, 10))
HR2 = np.reshape(n_HR, (766, 10))
IR2 = np.reshape(n_IR, (447, 10))
ITR2 = np.reshape(n_ITR, (1249, 10))

In [111]:
pca = PCA(n_components = 10)
pca2 = PCA(n_components = 3)

In [118]:
PC_ER = pca.fit_transform(ER2)
exp_ER = pca.explained_variance_ratio_
print(exp_ER)
components = pca2.fit_transform(ER2)
loadings = pca2.components_.T * np.sqrt(pca2.explained_variance_)
loading_matrix = pd.DataFrame(loadings, columns = ['PC1', 'PC2', 'PC3'], index = energy_stocks)
print(loading_matrix)

[0.75217645 0.07398827 0.04259949 0.03467262 0.0231421  0.02010241
 0.01668328 0.01432796 0.01134607 0.01096135]
           PC1       PC2       PC3
CVX  -0.839975 -0.216323  0.366168
XOM  -0.822298  0.021062 -0.361238
HES  -0.928569 -0.047592  0.005967
BKR  -0.575897  0.792899  0.173617
HAL  -0.900177  0.092575 -0.179294
SLB  -0.906700 -0.127992  0.184910
APA  -0.917313 -0.112872 -0.073724
COP  -0.894241 -0.101628  0.178285
CTRA -0.924623 -0.094407 -0.099214
DVN  -0.907569  0.073136 -0.134845


In [119]:
PC_FR = pca.fit_transform(FR2)
exp_FR = pca.explained_variance_ratio_
print(exp_FR)
components = pca2.fit_transform(FR2)
loadings = pca2.components_.T * np.sqrt(pca2.explained_variance_)
loading_matrix = pd.DataFrame(loadings, columns = ['PC1', 'PC2', 'PC3'], index = financial_stocks)
print(loading_matrix)

[0.77671677 0.06953378 0.04274259 0.02951582 0.02176818 0.01525232
 0.01330932 0.01151492 0.01082462 0.00882167]
           PC1       PC2       PC3
BLK   0.021600 -0.006031 -0.000280
BK    0.020057  0.008924 -0.004474
BEN   0.018935  0.001046  0.007204
IVZ   0.017797  0.004642 -0.000326
NTRS  0.024681 -0.008675 -0.002228
STT   0.026719 -0.010740 -0.004382
TROW  0.025343  0.007179 -0.006929
AXP   0.020401  0.001512  0.005963
COF   0.022112  0.001027  0.008992
DFS   0.018049  0.006341 -0.000423


In [120]:
PC_HR = pca.fit_transform(HR2)
exp_HR = pca.explained_variance_ratio_
print(exp_HR)
components = pca2.fit_transform(HR2)
loadings = pca2.components_.T * np.sqrt(pca2.explained_variance_)
loading_matrix = pd.DataFrame(loadings, columns = ['PC1', 'PC2', 'PC3'], index = health_stocks)
print(loading_matrix)

[0.46056757 0.25603764 0.10135808 0.05542461 0.03246763 0.02429093
 0.02354707 0.01992785 0.0147547  0.01162391]
           PC1       PC2       PC3
AMGN  0.014215 -0.010854 -0.007969
BIIB  0.014116 -0.009440 -0.003903
GILD  0.021149 -0.016777  0.024833
INCY  0.011772 -0.008669 -0.002784
MRNA  0.014679 -0.010152 -0.008762
REGN  0.014092 -0.005810 -0.004161
VRTX  0.015673 -0.009011 -0.002952
ABC   0.041103  0.034470  0.001261
BMY   0.016860 -0.008009 -0.003247
CAH   0.015537 -0.010634 -0.005639


In [121]:
PC_IR = pca.fit_transform(IR2)
exp_IR = pca.explained_variance_ratio_
print(exp_IR)
components = pca2.fit_transform(IR2)
loadings = pca2.components_.T * np.sqrt(pca2.explained_variance_)
loading_matrix = pd.DataFrame(loadings, columns = ['PC1', 'PC2', 'PC3'], index = industrial_stocks)
print(loading_matrix)

[0.61546875 0.12143353 0.07788201 0.05177938 0.03663594 0.03425559
 0.02144747 0.01873401 0.0138677  0.00849562]
           PC1       PC2       PC3
BA    0.043584  0.020111  0.004660
GD    0.032497  0.003406  0.001882
LMT   0.033921 -0.006487 -0.004889
RTX   0.012987 -0.015641  0.021940
FDX   0.012305 -0.007128 -0.006764
ALK   0.012570 -0.006576 -0.002839
AAL   0.012516 -0.009150 -0.003962
CARR  0.008983 -0.007463 -0.004711
MAS   0.008896 -0.009171 -0.003608
J     0.021422 -0.006092 -0.006556


In [123]:
PC_ITR = pca.fit_transform(ITR2)
exp_ITR = pca.explained_variance_ratio_
print(exp_ITR)
components = pca2.fit_transform(ITR2)
loadings = pca2.components_.T * np.sqrt(pca2.explained_variance_)
loading_matrix = pd.DataFrame(loadings, columns = ['PC1', 'PC2', 'PC3'], index = IT_stocks)
print(loading_matrix)

[0.63214789 0.11930508 0.08546287 0.04805389 0.03231022 0.02920376
 0.02407808 0.01677495 0.00841767 0.0042456 ]
           PC1       PC2       PC3
ADBE -0.017849  0.002965  0.003836
V    -0.019420  0.003246  0.006981
ORCL -0.027946 -0.022447 -0.003360
CRM  -0.018653  0.004196  0.006529
MA   -0.017597  0.006706  0.002229
AMD  -0.017615  0.003956  0.004588
NVDA -0.027206 -0.005249  0.000757
MSFT -0.014245  0.005003  0.003068
WDC  -0.016086  0.005998  0.002376
AAPL -0.022633  0.009740 -0.020080


In [124]:
def sharpe(stocks, rf, n_assets, returns):
    sharpe_ratios = {}
    for i, j in zip(stocks, range(n_assets)):
        sharpe_ratios[i] = (returns.iloc[:,j].mean() - rf) / np.std(returns.iloc[:,j])
    return sharpe_ratios

In [128]:
risk_free_mean = risk_free.mean()
sharpe_energy = sharpe(energy_prices_close.columns, risk_free_mean, 10, energy_returns.iloc[:, -10:])
sharpe_financial = sharpe(financial_prices_close.columns, risk_free_mean, 10, financial_returns.iloc[:, -10:])
sharpe_health = sharpe(health_prices_close.columns, risk_free_mean, 10, health_returns.iloc[:, -10:])
sharpe_industrial = sharpe(industrial_prices_close.columns, risk_free_mean, 10, industrial_returns.iloc[:, -10:])
sharpe_IT = sharpe(IT_prices_close.columns, risk_free_mean, 10, IT_returns.iloc[:, -10:])

In [129]:
sharpes = {'Energy':sharpe_energy, 'Financial':sharpe_financial, 'Health':sharpe_health,
           'Industrial':sharpe_industrial, 'IT':sharpe_IT}
sharpes

{'Energy': {'APA': -0.5356369741242483,
  'BKR': -0.7926599011035328,
  'COP': -0.8204082395440592,
  'CTRA': -0.9338760287285021,
  'CVX': -1.0227320118268657,
  'DVN': -0.6147358319480448,
  'HAL': -0.6977611842871301,
  'HES': -0.7181673939647232,
  'SLB': -0.8139325358046346,
  'XOM': -1.1023219089119423},
 'Financial': {'AXP': -0.962955110354781,
  'BEN': -0.9911954149016502,
  'BK': -1.1120455537498042,
  'BLK': -1.104978242074406,
  'COF': -0.860957685489293,
  'DFS': -0.783647455237164,
  'IVZ': -0.8308705282277895,
  'NTRS': -1.054971617628077,
  'STT': -0.9533820143373921,
  'TROW': -1.0848862709570972},
 'Health': {'ABC': -0.9034158736881408,
  'AMGN': -1.0271013057108875,
  'BIIB': -0.585309099618193,
  'BMY': -1.1006366964873964,
  'CAH': -0.8730621865018454,
  'GILD': -1.0458689654189357,
  'INCY': -0.860968121257213,
  'MRNA': -0.31775115131504567,
  'REGN': -0.8636071046129565,
  'VRTX': -0.8738050717420119},
 'Industrial': {'AAL': -0.21204216073850124,
  'ALK': -0.2882

In [130]:
sharpe_SP = sharpe(pd.DataFrame(SP_prices_close).columns, risk_free_mean, 1, pd.DataFrame(SP_returns).iloc[:, -1:])

In [131]:
sharpe_SP

{'Close': -1.4735006677653695}

The Sharpe ratios seem to indicate something similar, and to an even greater degree, that the risk is incredible (which makes sense, given the huge loss from the pandemic across all sectors)